In [1]:
import ccxt

import os
import time
import numpy as np
import json
import pandas as pd
import redis
import datetime
import decimal
import inspect
import sys

In [2]:
def round_down(value, decimals):
    with decimal.localcontext() as ctx:
        d = decimal.Decimal(value)
        ctx.rounding = decimal.ROUND_DOWN
        return float(round(d, decimals))


In [3]:
apiKey = os.getenv('binance_ratio_ID')
apiSecret = os.getenv('binance_ratio_SECRET')

In [4]:
r = redis.Redis(host='localhost', port=6379, db=0)   

In [5]:
exchange = ccxt.binance({
                        'apiKey': apiKey,
                        'secret': apiSecret,
                        'enableRateLimit': True,
                        'options': {'defaultMarket': 'futures'}
                    })

In [18]:
markets = exchange.load_markets()

In [6]:
balances = pd.DataFrame(exchange.fetch_balance()['info']['balances'])

In [7]:
balance = float(balances[balances['asset'] == 'BTC'].iloc[0]['free'])

In [8]:
exchange.sapi_post_margin_isolated_transfer({'asset': 'BTC', 'symbol': 'ETHBTC', 'transFrom': 'SPOT', 'transTo': 'ISOLATED_MARGIN', 'amount': balance})

BadRequest: binance {"code":-1102,"msg":"Mandatory parameter 'amount' was not sent, was empty/null, or malformed."}

In [9]:
exchange.sapi_post_margin_isolated_transfer({'asset': 'BTC', 'symbol': 'ETHBTC', 'transFrom': 'ISOLATED_MARGIN', 'transTo': 'SPOT', 'amount': balance})

{'tranId': 13633853843, 'clientTag': ''}

In [30]:
# open long
# exchange.sapi_post_margin_order({'symbol': 'ETHBTC', 'isIsolated': 'TRUE', 'side': 'BUY', 'type': 'MARKET', 'quantity': 0.1, 'sideEffectType': 'MARGIN_BUY'})

In [31]:
# close long
# exchange.sapi_post_margin_order({'symbol': 'ETHBTC', 'isIsolated': 'TRUE', 'side': 'SELL', 'type': 'MARKET', 'quantity': rounded, 'sideEffectType': 'AUTO_REPAY'})

In [ ]:
exchange.sapi_post_margin_order({'symbol': 'ETHBTC', 'isIsolated': 'TRUE', 'side': 'SELL', 'type': 'MARKET', 'quantity': 0.02, 'sideEffectType': 'MARGIN_BUY'})

In [32]:
exchange.sapi_post_margin_order({'symbol': 'ETHBTC', 'isIsolated': 'TRUE', 'side': 'BUY', 'type': 'LIMIT', 'quantity': 0.02, 'sideEffectType': 'MARGIN_BUY'})

{'symbol': 'ETHBTC',
 'orderId': 1184163264,
 'clientOrderId': 'default_39396d55025b42078c095c057fdb',
 'transactTime': 1609742360797,
 'price': '0',
 'origQty': '0.02',
 'executedQty': '0.02',
 'cummulativeQuoteQty': '0.00069808',
 'status': 'FILLED',
 'timeInForce': 'GTC',
 'type': 'MARKET',
 'side': 'SELL',
 'fills': [{'price': '0.034904',
   'qty': '0.02',
   'commission': '0.0000007',
   'commissionAsset': 'BTC'}],
 'marginBuyBorrowAsset': 'ETH',
 'marginBuyBorrowAmount': '0.01930254',
 'isIsolated': True}

In [54]:
def get_position(symbol):
    min_notional = float(exchange.markets['ETH/BTC']['info']['filters'][2]['stepSize'])
    details = exchange.sapi_get_margin_isolated_account({'symbols': symbol})['assets'][0]
    current_pos = "NONE"
    amount = 0
    avgEntryPrice = 0
    asset = ""

    if float(details['baseAsset']['borrowed']) == 0 and float(details['quoteAsset']['borrowed']) != 0:
        current_pos = "LONG"
        amount = float(details['baseAsset']['free'])

        if abs(amount) <= min_notional * 3:
            return "NONE", 0, "", 0

        asset = details['baseAsset']['asset']
        avgEntryPrice = float(exchange.sapi_get_margin_mytrades({'symbol' :symbol, 'isIsolated': 'TRUE'})[-1]['price'])

    elif float(details['baseAsset']['borrowed']) != 0 and float(details['quoteAsset']['borrowed']) == 0:
        current_pos = "SHORT"
        amount = float(details['baseAsset']['borrowed'])

        if abs(amount) <= min_notional * 3:
            return "NONE", 0, "", 0

        asset = details['baseAsset']['asset']
        avgEntryPrice = float(exchange.sapi_get_margin_mytrades({'symbol' :symbol, 'isIsolated': 'TRUE'})[-1]['price'])
        
    step = str(min_notional)
    amount = round_down(amount,len(step.split('.')[1]))
    
    return current_pos, amount, asset, avgEntryPrice

In [55]:
get_position('ETHBTC')

('NONE', 0, '', 0)

In [41]:
exchange.sapi_post_margin_order({'symbol': 'ETHBTC', 'isIsolated': 'TRUE', 'side': 'BUY', 'type': 'MARKET', 'quantity': 0.002, 'sideEffectType': 'AUTO_REPAY'})

InvalidOrder: binance {"code":-1013,"msg":"Filter failure: MIN_NOTIONAL"}

In [ ]:
#i think this works. now test, And make it close and open well